<a href="https://colab.research.google.com/github/divya-medarametla/CMPE-255-Credit_Hazard-Forecasting/blob/main/Credit_hazard_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import numpy as np
from IPython.display import display
import plotly.express as px
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_validate, train_test_split
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets, metrics, model_selection, svm

credit_approval = pd.read_csv("credit_risk_dataset.csv", header = None)
credit_approval.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
1,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
2,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.1,N,2
3,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
4,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2


In [13]:
import os

In [18]:
DATASET_DIR='dataset/'
DATASET_FILENAME='dataset_'

In [19]:
def read_dataset(file_path):
    """
    This function is used to read data from file and produce dataframe with file contents.
    
    Input: file_path
    
    Output: Dataframe with file contents
    
    """
    data_flag=False
    data=[]
    features=[]
    with open(file_path) as f:
        all_lines=f.readlines()
        for line in all_lines:
            if '@ATTRIBUTE' in line:
                _,feature_name,_=line.split(' ')
                features.append(feature_name)
            if data_flag:
                line=line.replace("\n",'')
                data.append(line.split(','))
            if '@DATA' in line:
                data_flag=True   
    df_data=pd.DataFrame(data=data,columns=features)
    return df_data


In [8]:

import numpy as np
credit_approval = credit_approval.replace("?", np.NaN)

In [ ]:

print(credit_approval[0].value_counts())
credit_approval[0] = credit_approval[0].fillna(credit_approval[0].value_counts().index[0])
credit_approval[0].tail(5)

23            3889
22            3633
24            3549
25            3037
26            2477
27            2138
28            1854
29            1687
30            1316
21            1229
31            1142
32             964
33             856
34             709
35             620
36             548
37             478
38             373
39             302
40             271
41             241
42             188
43             164
44             141
45             108
47              94
46              94
48              75
50              52
49              49
51              39
52              36
53              30
54              24
55              20
58              19
56              15
60              15
20              15
57              15
66               9
61               9
65               9
62               7
64               7
70               7
59               5
69               5
144              3
73               3
63               3
123              2
80          

32577    57
32578    54
32579    65
32580    56
32581    66
Name: 0, dtype: object

In [ ]:

print(credit_approval[3].value_counts())
credit_approval[3] = credit_approval[3].fillna(credit_approval[3].value_counts().index[0])
credit_approval[3].tail(5)

0.0                  5000
2.0                  3849
3.0                  3456
5.0                  2946
1.0                  2915
4.0                  2874
6.0                  2666
7.0                  2196
8.0                  1687
9.0                  1367
11.0                  740
10.0                  696
12.0                  575
13.0                  426
14.0                  335
15.0                  238
16.0                  165
17.0                  129
18.0                  104
19.0                   64
20.0                   42
21.0                   38
22.0                   19
24.0                   10
23.0                   10
25.0                    8
26.0                    6
27.0                    5
31.0                    4
28.0                    3
30.0                    2
123.0                   2
41.0                    1
34.0                    1
29.0                    1
38.0                    1
person_emp_length       1
Name: 3, dtype: int64


32577    1.0
32578    4.0
32579    3.0
32580    5.0
32581    2.0
Name: 3, dtype: object

In [ ]:

print(credit_approval[4].value_counts())
credit_approval[4] = credit_approval[4].fillna(credit_approval[4].value_counts().index[0])
credit_approval[4].tail(5)

EDUCATION            6453
MEDICAL              6071
VENTURE              5719
PERSONAL             5521
DEBTCONSOLIDATION    5212
HOMEIMPROVEMENT      3605
loan_intent             1
Name: 4, dtype: int64


32577           PERSONAL
32578           PERSONAL
32579    HOMEIMPROVEMENT
32580           PERSONAL
32581            MEDICAL
Name: 4, dtype: object

In [ ]:

print(credit_approval[5].value_counts())
credit_approval[5] = credit_approval[5].fillna(credit_approval[5].value_counts().index[0])
print(credit_approval[5].isnull().values.sum())

A             10777
B             10451
C              6458
D              3626
E               964
F               241
G                64
loan_grade        1
Name: 5, dtype: int64
0


In [ ]:

print(credit_approval[6].value_counts())
print(credit_approval[6].isnull().values.sum())
credit_approval[6] = credit_approval[6].fillna(credit_approval[6].value_counts().index[0])
credit_approval[6].tail(5)

10000    2664
5000     2043
12000    1809
6000     1801
15000    1502
         ... 
29175       1
27300       1
17325       1
9625        1
29850       1
Name: 6, Length: 754, dtype: int64
0


32577     5800
32578    17625
32579    35000
32580    15000
32581     6475
Name: 6, dtype: object

In [ ]:

from sklearn.preprocessing import MinMaxScaler
def preprocess(data):
  scaler = MinMaxScaler(feature_range=(0, 1))
  return scaler.fit_transform(data)

In [ ]:

credit_approval.to_csv('crx_clean.data')
credit_approval.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,58,person_income,person_home_ownership,36,6,7,753,loan_int_rate,2,77,cb_person_default_on_file,29
1,4,59000,RENT,5,4,3,496,16.02,1,59,Y,21
2,3,9600,OWN,31,1,1,0,11.14,0,10,N,10
3,7,9600,MORTGAGE,1,3,2,575,12.87,1,57,N,21
4,5,65500,RENT,29,3,2,496,15.23,1,53,N,10
5,6,54400,RENT,34,3,2,496,14.27,1,55,Y,23
6,3,9900,OWN,13,5,0,408,7.14,1,25,N,10
7,8,77100,RENT,34,1,1,496,12.42,1,45,N,21
8,6,78956,RENT,31,3,1,496,11.11,1,44,N,23
9,6,83000,RENT,34,4,0,496,8.9,1,42,N,10
